In [1]:
import pandas as pd
import re 
import numpy as np

In [2]:
# Specify the file path to your CSV file and the encoding
csv_file_path = "attacks.csv"
csv_encoding = "latin-1"  # You can also try "ISO-8859-1"

# Read the CSV file into a DataFrame with the specified encoding
df = pd.read_csv(csv_file_path, encoding=csv_encoding)

# Now, 'df' contains your data from the CSV file

In [3]:
file_list = [f for f in df]
print(file_list)

['Case Number', 'Date', 'Year', 'Month', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order']


In [4]:
df.rename(columns = {"Sex ":"Sex"}, inplace = True)
df.rename(columns = {"Species ":"Species"}, inplace = True)

In [5]:
df = df.drop(["Date","Name","Investigator or Source","Activity","pdf","href formula","href","original order","Case Number.1", "Case Number.2"], axis = 1)

In [6]:
print(df.shape)
print(df.dtypes)


(6094, 13)
Case Number    object
Year            int64
Month           int64
Type           object
Country        object
Area           object
Location       object
Sex            object
Age             int64
Injury         object
Fatal (Y/N)    object
Time           object
Species        object
dtype: object


In [7]:
null_cols = df.isnull().sum()
null_cols

Case Number       0
Year              0
Month             0
Type              0
Country          21
Area            296
Location        369
Sex               0
Age               0
Injury           19
Fatal (Y/N)       0
Time           2654
Species        2435
dtype: int64

In [8]:
df["Year"] = df["Year"].astype(str)

In [9]:
df['Country'] = df['Country'].str.replace('England', 'UNITED KINGDOM')
df['Country'] = df['Country'].str.replace('SCOTLAND', 'UNITED KINGDOM')
df['Country'] = df['Country'].str.replace('ENGLAND', 'UNITED KINGDOM')

filtered = df[(df['Country'] == "UNITED KINGDOM")]
print(filtered.shape)

(32, 13)


In [10]:
df["Type"] = df["Type"].str.replace('Unknown', 'Unspecified')
df["Type"] = df["Type"].str.replace('Sea Disaster', 'Unprovoked')
df.loc[df['Type'].str.startswith('Boat'), "Type"] = 'Provoked'

df["Type"].value_counts()

Unprovoked     4134
Provoked        816
0               683
Unspecified     461
Name: Type, dtype: int64

In [11]:
dict_time = {'':['>','<','h','Before','Early','Earlyafter', ' ','Late','Mid','Mid-','j','After', 'Lateafter','after'],
            'Morning':['-Morning','A.M.','AM','PM','Sometimebetween0600&08oo','2rsbeforesunset','morning','Between 1100 & 1200'],
            'Evening':['"evening"','"Evening"','Sunset','nigt','Nigt','P.M.','Dusk','"sortly before dusk"'],
            'Afternoon':['Noon','Late afternon','Late afternoon','Early afternoon','noon','Midday.','Midday','Sortlybefore1300','1101 -time of sip sinking'],
            'Unspecified':['day','Possiblysameincidentas2000.08.21','FATAL  (Wire netting installed at local beaces after tis incident.)','2 rs before sunset','FATAL(Wire netting installed at local beaces after tis incident.)','--']}

dict_time_s = {'Early Morning':['00','01','02','03','04','05'],
               'Morning':['06','07','08','09','10','11'],
              'Afternoon':['12','13','14','15','16','17'],
              'Evening':['18','19','20','21','22','23','00','01','02','03','04','05']}


def stWdict(dictionary, dataframe, column):
    #startwith
    for k,v in dictionary.items():
        for e in v: 
            dataframe.loc[dataframe[column].str.startswith(e), column] = k
    return dataframe



def repDict(dictionary, dataframe, column):
    #replace 
    for k,v in dictionary.items():
        for e in v:
            dataframe[column] = dataframe[column].str.replace(e,k)  
    
    return dataframe 

In [12]:
repDict(dict_time, df, 'Time')
df['Time'] = df['Time'].fillna("Unspecified")

/var/folders/j5/_42xd4zd3mg_z3qdfvj11tjw0000gn/T/ipykernel_17175/3070096504.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[column] = dataframe[column].str.replace(e,k)


In [13]:
stWdict(dict_time_s, df, 'Time')

,Case Number,Year,Month,Type,Country,Area,Location,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,2017.06.11,2017,6,Unprovoked,AUSTRALIA,Western Australia,"Point Casuarina, Bunbury",M,48,"No injury, board bitten",N,Morning,"White shark, 4 m"
1,2017.06.10.b,2017,6,Unprovoked,AUSTRALIA,Victoria,"Flinders, Mornington Penisula",F,22,"No injury, knocke off board",N,Afternoon,7 gill shark
2,2017.06.10.a,2017,6,Unprovoked,USA,Florida,"Ponce Inlet, Volusia County",M,19,Laceration to left foot,N,Morning,NaN
3,2017.06.07.R,2017,6,Unprovoked,UNITED KINGDOM,South Devon,Bantham Beach,M,30,"Bruise to leg, cuts to hand sustained when he ...",N,Unspecified,"3m shark, probably a smooth hound"
4,2017.06.04,2017,6,Unprovoked,USA,Florida,"Middle Sambo Reef off Boca Chica, Monroe County",M,22,Laceration to shin,N,Unspecified,8' shark
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6089,ND.0005,0,0,0,0,0,0,0,0,0,0,0,0
6090,ND.0004,0,0,0,0,0,0,0,0,0,0,0,0
6091,ND.0003,0,0,0,0,0,0,0,0,0,0,0,0
6092,ND.0002,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df['Time'].value_counts()

Unspecified                                             2669
Afternoon                                               1479
Morning                                                  863
0                                                        683
Evening                                                  324
Early Morning                                             32
                                                           4
non                                                        2
JustbeforeAfternoon                                        2
Dawn                                                       2
Dark                                                       2
Unspecified.                                               1
-Morning                                                   1
Justbeforedawn                                             1
Daytime                                                    1
"lunc"                                                     1
Between1100&1200        

## Note: Remaining Cleaning for time will be done manually on Excel

In [15]:
df["Area"]= df["Area"].fillna("Unspecified")

In [16]:
df["Area"].value_counts()

Florida                                           987
0                                                 683
New South Wales                                   413
Unspecified                                       296
Queensland                                        277
                                                 ... 
Middle Caicos                                       1
Western Caroline Islands (North Pacific Ocean)      1
Namonuito Atoll                                     1
San André archipelago                             1
Baja California Sur                                 1
Name: Area, Length: 700, dtype: int64

In [17]:
count = df['Area'].value_counts()
count[count > 10]
df=df[df['Area'].isin(count[count >10].index)]
df["Area"].value_counts()

Florida                  987
0                        683
New South Wales          413
Unspecified              296
Queensland               277
California               275
Hawaii                   267
KwaZulu-Natal            197
Western Cape Province    183
Western Australia        165
Eastern Cape Province    150
South Carolina           133
South Australia           92
North Carolina            92
Pernambuco                71
Victoria                  69
Texas                     67
Torres Strait             65
North Island              46
New Jersey                42
Tasmania                  33
South Island              31
Oregon                    29
Abaco Islands             22
Central Province          22
Northern Territory        21
Puerto Rico               16
Gaza                      16
Virginia                  16
South Province            15
New Ireland Province      15
New York                  14
Madang Province           14
Veracruz                  14
Guerrero      

In [18]:
!pip install countryinfo

In [19]:
from countryinfo import CountryInfo
from subprocess import check_output
import seaborn as sns

def clean_colnames(df):
    '''Clean columns names when passing a pandas dataframe: params (df - dataframe)'''
    col_clean = []
    for col in df.columns:
        col = col.strip().lower()
        col = col.replace('.',' ')
        col_clean.append(col)
        
    df.columns = col_clean
    return df.columns

def change_col_pos(col,pos,df):
    '''Change the position of a column in a dataframe: params(col - column name, 
    pos - new index position, df - dataframe)'''
    all_cols = df.columns.tolist()
    temp = all_cols.pop(all_cols.index(col))
    all_cols.insert(pos,col)
    
    df = df[all_cols]
    
    return df

def country_hem(col):
    '''Return a list if countries are part of the north (1) or south (0) hemispheres of planet earth. If country
    is not recognized, NaN is returned'''
    hem_lst = []
    for row in col:
        try:
            country = CountryInfo(row)
        except AttributeError:
            row = 'x'
   
        try:
            pos = country.latlng()
        except KeyError:
            pos = (0,0)
    
        if pos[0] > 0:
            temp_row = 1
        elif pos[0] < 0:
            temp_row = 0
        else:
            temp_row = np.nan
    
        hem_lst.append(temp_row)
    
    return hem_lst

In [20]:
df['Country'].unique()

array(['AUSTRALIA', 'USA', 'SOUTH AFRICA', 'NEW ZEALAND', 'BRAZIL',
       'MEXICO', 'NEW CALEDONIA', 'REUNION', 'BAHAMAS', 'ARUBA', 'FIJI',
       'PUERTO RICO', 'EGYPT', 'ATLANTIC OCEAN', 'GREECE', 'JAPAN', nan,
       'ST. MARTIN', 'CAYMAN ISLANDS', 'PAPUA NEW GUINEA', 'DIEGO GARCIA',
       'PALESTINIAN TERRITORIES', 'GUAM', 'SEYCHELLES', 'BELIZE',
       'CROATIA', 'PHILIPPINES', 'SAMOA', 'DOMINICAN REPUBLIC', 'AZORES',
       'MALTA', 'VIETNAM', 'SOMALIA', 'NEVIS', 'TAIWAN', 'SENEGAL',
       'COSTA RICA', 'ITALY', 'THAILAND', 'LIBERIA', 'MADAGASCAR',
       'VENEZUELA', 'INDONESIA', 'TURKS & CAICOS', 'CARIBBEAN SEA',
       'TANZANIA', 'MAURITIUS', 'MOZAMBIQUE', 'SOUTH KOREA',
       'NORTHERN ARABIAN SEA', 'CHILE', 'HONG KONG', 'BERMUDA', 'IRAN',
       'TUNISIA', 'UNITED KINGDOM', 'NORTH ATLANTIC OCEAN',
       'SOUTH CHINA SEA', 'PACIFIC OCEAN ', 'SOLOMON ISLANDS', 'TONGA',
       'COLUMBIA', 'SINGAPORE', 'JOHNSTON ISLAND', 'SOUTH PACIFIC OCEAN',
       'JAMAICA', 'NEW GUINEA

In [21]:
for row in df['Country']:
    if isinstance(row, str):
        new_row = re.sub('\/.+|\(.+\)|\.|\?', '', row)
        new_row = re.sub('\&', 'and', new_row.strip().upper())
        
        if new_row == 'USA':
            new_row = new_row.replace(new_row, 'UNITED STATES')
        elif new_row == 'BAHAMAS':
            new_row = new_row.replace(new_row, 'THE BAHAMAS')
        elif new_row == 'BRITISH ISLES':
            new_row = new_row.replace(new_row, 'UNITED KINGDOM')
        elif new_row == 'REUNION':
            new_row = new_row.replace(new_row, 'réunion')
        elif new_row == 'OKINAWA':
            new_row = new_row.replace(new_row, 'JAPAN')
        elif new_row == 'AZORES':
            new_row = new_row.replace(new_row, 'PORTUGAL')
        elif new_row == 'RED SEA':
            new_row = new_row.replace(new_row, 'EGYPT')
        elif new_row == 'COLUMBIA':
            new_row = new_row.replace(new_row, 'COLOMBIA')
        elif new_row == 'NEW BRITAIN' or new_row == 'NEW GUINEA' or new_row == 'BRITISH NEW GUINEA' or new_row == 'ADMIRALTY ISLANDS':
            new_row = new_row.replace(new_row, 'PAPUA NEW GUINEA')
        
        df['Country'].replace(row,new_row, inplace = True)
    else:
        df['Country'].replace(row,np.nan, inplace = True)

/var/folders/j5/_42xd4zd3mg_z3qdfvj11tjw0000gn/T/ipykernel_17175/2696976617.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'].replace(row,new_row, inplace = True)
/var/folders/j5/_42xd4zd3mg_z3qdfvj11tjw0000gn/T/ipykernel_17175/2696976617.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'].replace(row,np.nan, inplace = True)


In [22]:
df['Country'].unique()

array(['AUSTRALIA', 'UNITED STATES', 'SOUTH AFRICA', 'NEW ZEALAND',
       'BRAZIL', 'MEXICO', 'NEW CALEDONIA', 'RÉUNION', 'THE BAHAMAS',
       'ARUBA', 'FIJI', 'PUERTO RICO', 'EGYPT', 'ATLANTIC OCEAN',
       'GREECE', 'JAPAN', nan, 'ST MARTIN', 'CAYMAN ISLANDS',
       'PAPUA NEW GUINEA', 'DIEGO GARCIA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'CROATIA', 'PHILIPPINES', 'SAMOA',
       'DOMINICAN REPUBLIC', 'PORTUGAL', 'MALTA', 'VIETNAM', 'SOMALIA',
       'NEVIS', 'TAIWAN', 'SENEGAL', 'COSTA RICA', 'ITALY', 'THAILAND',
       'LIBERIA', 'MADAGASCAR', 'VENEZUELA', 'INDONESIA',
       'TURKS and CAICOS', 'CARIBBEAN SEA', 'TANZANIA', 'MAURITIUS',
       'MOZAMBIQUE', 'SOUTH KOREA', 'NORTHERN ARABIAN SEA', 'CHILE',
       'HONG KONG', 'BERMUDA', 'IRAN', 'TUNISIA', 'UNITED KINGDOM',
       'NORTH ATLANTIC OCEAN', 'SOUTH CHINA SEA', 'PACIFIC OCEAN',
       'SOLOMON ISLANDS', 'TONGA', 'COLOMBIA', 'SINGAPORE',
       'JOHNSTON ISLAND', 'SOUTH PACIFIC OCEAN', 'JAMAI

In [23]:
# Filling NaN value with string "Unknown shark"
df['Species'] = df['Species'].fillna('Unknown shark')

# Create new column to store the extracted Shark species
df['New Species'] = None

# Regular Expression pattern used to extract shark species (ultimately getting everything infront of the word "shark")
shark_pattern = r'.* (shark|Shark)'

# Looping through every row in the dataFrame
for row in range(len(df)):
    try:
        shark_species = re.search(shark_pattern, df.iat[row, df.columns.get_loc('Species')]).group()
        df.iat[row, df.columns.get_loc('New Species')] = shark_species
    except:
        # Assuming that if there is nothing before string shark OR doesn't contain string shark at all,
        # then I will put the string Unconfirmed Shark involvement
        df.iat[row, df.columns.get_loc('New Species')] = "Unconfirmed Shark involvement"

bySpecies_count = df['New Species'].value_counts().reset_index().rename(columns={'New Species':'Count','index':'Species'})
bySpecies_count.head()

/var/folders/j5/_42xd4zd3mg_z3qdfvj11tjw0000gn/T/ipykernel_17175/963116223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Species'] = df['Species'].fillna('Unknown shark')
/var/folders/j5/_42xd4zd3mg_z3qdfvj11tjw0000gn/T/ipykernel_17175/963116223.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['New Species'] = None


,Species,Count
0,Unknown shark,1839
1,Unconfirmed Shark involvement,833
2,White shark,345
3,Tiger shark,185
4,Bull shark,87


In [24]:
df['hemisphere'] = country_hem(df['Country'])

/var/folders/j5/_42xd4zd3mg_z3qdfvj11tjw0000gn/T/ipykernel_17175/4265330549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hemisphere'] = country_hem(df['Country'])


In [25]:
df = change_col_pos('hemisphere',6,df)

In [26]:
df['hemisphere'].value_counts(dropna=False)

1.0    2207
0.0    2016
NaN     731
Name: hemisphere, dtype: int64

In [27]:
nh = ['north sea', 'scotland', 'north pacific ocean', 'turks and caicos', 'caribbean sea', 'persian gulf', 'micronesia', 
     'burma', 'north atlantic ocean', 'montenegro', 'the balkans', 'northern arabian sea', 'netherlands antilles', 'mediterranean sea',
     'grand cayman', 'netherlands antilles', 'south china sea', 'st martin', 'andaman', 'palestinian territories', 'johnston island',
     'nevis', 'bay of bengal']
sh = ['south atlantic ocean', 'western samoa', 'southwest pacific ocean', 'tasman sea', 'st helena']

In [28]:
for row in df.index:
    if df.at[row,'Country'] in nh:
        df.at[row, 'hemisphere'] = 1
    elif df.at[row, 'Country'] in sh:
        df.at[row, 'hemisphere'] = 0

In [29]:
df['hemisphere'].value_counts(dropna=False)

1.0    2207
0.0    2016
NaN     731
Name: hemisphere, dtype: int64

In [30]:
df[df['Country'] == 'atlantic ocean']

,Case Number,Year,Month,Type,Country,Area,hemisphere,Location,Sex,Age,Injury,Fatal (Y/N),Time,Species,New Species


In [31]:
df.at[4561, 'hemisphere'] = 0

In [32]:
df["Month"] = df["Month"].astype(str)

In [33]:
n_seasons = {'Spring': ['3', '4', '5'],
             'Summer': ['6', '7', '8'],
             'Autumn': ['9', '10', '11'],
             'Winter': ['12', '1', '2']}

s_seasons = {'Spring': ['9', '10', '11'],
             'Summer': ['12', '1', '2'],
             'Autumn': ['3', '4', '5'],
             'Winter': ['6', '7', '8']}

temp_seasons = []

# Define a function to determine the season for a given row
def get_season(row):
    if row['hemisphere'] == 1:
        if row['Month'] in n_seasons['Spring']:
            return 'Spring'
        elif row['Month'] in n_seasons['Summer']:
            return 'Summer'
        elif row['Month'] in n_seasons['Autumn']:
            return 'Autumn'
        elif row['Month'] in n_seasons['Winter']:
            return 'Winter'
    
    elif row['hemisphere'] == 0:
        if row['Month'] in s_seasons['Spring']:
            return 'Spring'
        elif row['Month'] in s_seasons['Summer']:
            return 'Summer'
        elif row['Month'] in s_seasons['Autumn']:
            return 'Autumn'
        elif row['Month'] in s_seasons['Winter']:
            return 'Winter'
    
    return 'Unknown'  # Default value for unmatched rows

# Apply the get_season function to each row in the DataFrame
df['seasons'] = df.apply(get_season, axis=1)

In [34]:
# Define regular expressions for body parts
upper_body_pattern = r"(head|neck|chest|shoulder|back|arm|hand|wrist|shin|Upper Body|abdomen|torso)"
lower_body_pattern = r"(leg|hip|thigh|knee|calf|ankle|foot|buttocks|Lower Body|toes)"
multiple_parts_pattern = r"(head\s*,|\s*neck\s*,|\s*chest\s*,|\s*shoulder\s*,|\s*back\s*,|" \
                         r"\s*arm\s*,|\s*hand\s*,|\s*wrist\s*,|\s*leg\s*,|\s*hip\s*,|" \
                         r"\s*thigh\s*,|\s*knee\s*,|\s*calf\s*,|\s*ankle\s*,|\s*foot\s*|\s*Multiple Injuries\s*|\s*torso\s*)"

# Function to categorize injuries
def categorize_injuries(description):
    if isinstance(description, str):
        if re.search(upper_body_pattern, description, re.I):
            return "Upper Body"
        elif re.search(lower_body_pattern, description, re.I):
            return "Lower Body"
        elif re.search(multiple_parts_pattern, description, re.I):
            return "Multiple Parts"
        else:
            return "No Injuries"
    else:
        return "No Injuries"  # Handle non-string values

# Apply categorization to the 'Injury' column and store the result in 'Injury_Category' column
df['Injury_Category'] = df['Injury'].apply(categorize_injuries)

# Print the DataFrame with the new column
print(df)

       Case Number  Year Month        Type        Country               Area  \
0       2017.06.11  2017     6  Unprovoked      AUSTRALIA  Western Australia   
1     2017.06.10.b  2017     6  Unprovoked      AUSTRALIA           Victoria   
2     2017.06.10.a  2017     6  Unprovoked  UNITED STATES            Florida   
4       2017.06.04  2017     6  Unprovoked  UNITED STATES            Florida   
6       2017.05.30  2017     5    Provoked  UNITED STATES     South Carolina   
...            ...   ...   ...         ...            ...                ...   
6089       ND.0005     0     0           0              0                  0   
6090       ND.0004     0     0           0              0                  0   
6091       ND.0003     0     0           0              0                  0   
6092       ND.0002     0     0           0              0                  0   
6093       ND.0001     0     0           0              0                  0   

      hemisphere                       

In [35]:
# Define the output Excel file path
output_excel_file = 'output_data.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(output_excel_file, index=False)

print(f'Data has been saved to {output_excel_file}')

Data has been saved to output_data.xlsx
